## **First Question Answer Reason**

- **A.** `"Your friend's next class is computer science."`  
  *Because the context clearly provides the answer, and the agent is instructed to use it.*

- **B.** `"I'm sorry, I do not have access to your friend's schedule."`  
  *Because plain string context is not automatically parsed or structured in the Agent SDK.*

- **C.** *Runtime error due to invalid context type.*  
  *Because the SDK only accepts dataclass-based context objects.*

- **D.** `"The agent answeringagent is not valid because it was not defined with a context model."`  
  *Because you didn't specify `Agent[str]` or `Agent[UserContext]`.*

# **Correct Answer: B**

## **Corrected Version of the code of question 1**

In [2]:
from openai import OpenAI
from agents.run import RunConfig
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, RunContextWrapper, function_tool
import asyncio
from dataclasses import dataclass
from pydantic import BaseModel, Field

In [ ]:
gemini_api_key = "your_gemini_api_key_here"  # Replace with your actual Gemini API key
client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=client
)

config = RunConfig(
    model=model,
    model_provider=client,
    tracing_disabled=True
)

In [14]:




@dataclass
class FriendClass():
    friend_name:str
    class_name:str



## code:
async def main():
    ctx = FriendClass(friend_name="Umme-habiba", class_name="PIAIC quarter two class")
    
    @function_tool
    async def fetch_friends_data(wrapper: RunContextWrapper[FriendClass]) -> str:
        """
            Returns the friends name and class by accessing the context object.
            No arguments are required.
        """
        return f"{wrapper.context.friend_name} is my friend having {wrapper.context.class_name} class."

        
    answeringagent = Agent[FriendClass](
        name="answering_agent",
        handoff_description="Specialist agent for answering questions about friends' classes.",
        instructions=(
            "You are given a context object with fields 'friend_name' and 'class_name'. "
            "Use the tool 'fetch_friends_data' to answer questions about this context."
            "If asked about the friends class, use the available tool to fetch it. do not ask it just fetch the data The tool works using context only and does not take user input"
        ),
        model=model,
        tools=[fetch_friends_data],
    )
    Triage_agent = Agent[FriendClass](
        name="orchestrator_agent",
        instructions="You are going to handoff the task to the appropriate agent.",
        handoffs=[answeringagent],
        model=model,
    )
    
        
    answer = await Runner.run(Triage_agent, "Tell me what is the class my friend Umme habiba will attend today? ", context=ctx )
    print(answer.final_output)

if __name__ == "__main__":
    await main()



Umme-habiba is your friend and she has PIAIC quarter two class.



In [14]:
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo:  
    name: str
    uid: int

@function_tool
async def fetch_user_age(wrapper: RunContextWrapper[UserInfo]) -> str: 
    """
        Returns the user's age by accessing the context object.
        No arguments are required.
    """ 
    return f"User {wrapper.context.name} is 47 years old"

async def main():
    user_info = UserInfo(name="John", uid=123)

    agent = Agent[UserInfo](  
        name="Assistant",
        instructions="If asked about the user's age, use the available tool to fetch it. The tool works using context only and does not take user input",
        tools=[fetch_user_age],
        model=model,
    )

    result = await Runner.run(  
        starting_agent=agent,
        input="What is the age of the user?",
        context=user_info,
    )

    print(result.final_output)  
    # The user John is 47 years old.

if __name__ == "__main__":
    await main()

The user is 47 years old.



### **Important things that I learned about context management in between agents**

- 1 thing is that agent does not knows how to transfer string as context to the agnets therefore we should make dataclass
- data classes are made to let the agents know that the context you are going to recive would be of this type
- also the type of context is not enough for the agent to understand how to use the agent we will have to use a function that has the RunContextWrapper class as a type of the input and gives str output so that the agent can use it and answer the question
- Also the agent should know that they will not be needing to input the arguments because that is directly given by the class called RunContextWrapper to the function and the agent can directly call it
- Runner.run or any function when will have the context it should be the type given to the agent with agent_name[T] of any type made by dataclass and make sure the information are filled in the required variables of the custom dataclass you made

### Question 2 **Code does not works!**

In [ ]:
async def main():
    
    # there is no function_tool decorator here, so the tool will not be used. and the agent
    # does not knows how to call the tool.
    async def answer_question() -> str:
        """If the agent can not answer the question, it will use this tool to answer it."""
        print("Answering question using the tool...")

    agent = Agent(
        model=model,
        name="example_agent",
        instructions="You are an agent that can answer questions about a specific topic.",
        tools=[answer_question]
    )
    
    result = await Runner.run(agent, "What is my father's name?")
    
    print(result.final_output)
    # The tool will be used to answer the question.
if __name__ == "__main__":
    await main()

### *Two Better Ways*

In [ ]:
from typing import Any
"""
    The first is obvious just use the function decorator tool
"""

""" 
    The second way is to import the FunctionTool class and use it directly. because
    the function_tool decorator also analyzes the function signatures it into the FunctionTool class
    to make it useable.
"""

## code:
from agents import FunctionTool

async def main():
    class AnswerQuestion(BaseModel):
        question: str = Field(..., description="The question to answer.")
    
    async def answer_question(context :RunContextWrapper[Any], *args) -> str:
        """A tool to answer questions. If the agent can not answer the question, it will use this tool to answer it."""
        parsed = AnswerQuestion.model_validate_json(args)
        return f"You asked: {parsed.question}"


    answering_tool = FunctionTool(
        name="answer_question",
        description="A tool to answer questions. call it whenever the agent does not know tha answer. It also does not needs any arguments.S",
        params_json_schema=AnswerQuestion.model_json_schema(),
        on_invoke_tool=answer_question,
    )
    
    agent = Agent(
        model=model,
        name="example_agent",
        instructions="You are an agent that will only call the tool and answer the users question.",
        tools=[answering_tool]
    )
    
    result = await Runner.run(agent, "What is my father's name?")
    
    print(result.final_output)
    # The tool will be used to answer the question.
if __name__ == "__main__":
    await main()


In [16]:
from typing import Any

from pydantic import BaseModel

from agents import RunContextWrapper, FunctionTool



def do_some_work(data: str) -> str:
    return "done"


class FunctionArgs(BaseModel):
    username: str
    age: int

async def main():
    async def run_function(ctx: RunContextWrapper[Any], args: str) -> str:
        parsed = FunctionArgs.model_validate_json(args)
        return do_some_work(data=f"{parsed.username} is {parsed.age} years old")


    tool = FunctionTool(
        name="process_user",
        description="Processes extracted user data",
        params_json_schema=FunctionArgs.model_json_schema(),
        on_invoke_tool=run_function,
    )
    
    agent_tools = Agent[FunctionArgs](
        model=model,
        name="example_agent",
        instructions="You are an agent that can process user data. and answer questions about it. You can use the tool 'process_user' to process the user data.",
        tools=[tool]
    )
    
    result = await Runner.run(
        agent_tools,
        "Process the user data for John and tell me his age.",
        context=RunContextWrapper(context=FunctionArgs(username="John", age=30))
    )
    print(result.final_output)

In [17]:
if __name__ == "__main__":
    await main()

InternalServerError: Error code: 503 - [{'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}]

NameError: name 'result' is not defined

print(result.final_output)